In [1]:
from gurobipy import GRB
import gurobipy as gb

# The parameters
oat_yield = 4.25
maize_yield = 3.0
soybean_yield = 20.0

# Number of options
CROPS = 3
PURCHASED = 2
SOLD = 4

# Selling prices
sell = [220, 260, 55, 26]
purchase = [264, 312]

# Create a new optimization model to maximize profit
model = gb.Model("Farming Problem")

# Construct the decision variables.
x = model.addVars(3, lb=0, vtype=GRB.CONTINUOUS, name="Crops")
w = model.addVars(4, lb=0, vtype=GRB.CONTINUOUS, name="Sold")
y = model.addVars(2, lb=0, vtype=GRB.CONTINUOUS, name="Purchased")

# Objective Function
model.setObjective(gb.quicksum(w[i]*sell[i] for i in range(SOLD)) - gb.quicksum(y[i]*purchase[i] for i in range(PURCHASED)), GRB.MAXIMIZE)

# Land capacity constraints 
land_constraint = model.addConstr(x[0] + x[1] + x[2] <= 500, "Land Capacity")

# Cattle feed constraints (oats)
cattle_constraint = model.addConstr(oat_yield*x[0] + y[0] - w[0] >= 200, "Oats")

# Cattle feed constraints (Maize)
oat_constraint = model.addConstr(maize_yield*x[1] + y[1] - w[1] >= 260, "Oats")

# Quota constraints (Soybean)
model.addConstr(w[2] <= 7000, "Quota")
soy_constraint = model.addConstr(w[2] + w[3] == soybean_yield*x[2], "Soybean")

# Solve our model
model.optimize()

# Append the objective function value
print("The optimal solution: ", model.objVal)
    
# Check if the optimization was successful
if model.status == gb.GRB.OPTIMAL:
    # Print the sensitivity analysis for the amount sold
    print("Optimal Amount Sold:")
    print(f"{'Oats'} = {w[0].x, w[0].RC, sell[0], w[0].SAObjUp, w[0].SAObjLow}")
    print(f"{'Maize'} = {w[1].x, w[1].RC, sell[1], w[1].SAObjUp, w[1].SAObjLow}")
    print(f"{'Soybean'} = {w[2].x, w[2].RC, sell[2], w[2].SAObjUp, w[2].SAObjLow}")
    print(f"{'Soybean'} = {w[3].x, w[3].RC, sell[3], w[3].SAObjUp, w[3].SAObjLow}")
else:
    print("Optimization was not successful.")

# Print sensitivity information
print("")
print(f"Sensitivity Information for Land Capacity Constraint {land_constraint.pi:.2f}:")
print("(LHS, RHS, Slack): ", (model.getRow(land_constraint).getValue(), land_constraint.RHS, land_constraint.slack))
print("Shadow Price: ", land_constraint.pi)
print("Range of Feasibility: ", (land_constraint.SARHSUp, land_constraint.SARHSLow))


Set parameter Username
Set parameter LicenseID to value 2610025
Academic license - for non-commercial use only - expires 2026-01-14
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[x86] - Darwin 24.2.0 24C101)

CPU model: Intel(R) Core(TM) i5-8257U CPU @ 1.40GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 5 rows, 9 columns and 13 nonzeros
Model fingerprint: 0x34930c79
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [3e+01, 3e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+02, 7e+03]
Presolve removed 2 rows and 2 columns
Presolve time: 0.02s
Presolved: 3 rows, 7 columns, 9 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    7.6800000e+33   2.000000e+30   7.680000e+03      0s
       4    4.0021667e+05   0.000000e+00   0.000000e+00      0s

Solved in 4 iterations and 0.06 seconds (0.00 work units)
Optimal objective  4.002166667e+05
The optimal solution

In [2]:

import gurobipy as gb

# Create a new optimization model for the dual problem
model = gb.Model("Dual Problem")

# Dual variables
y1 = model.addVar(lb=0, vtype=gb.GRB.CONTINUOUS, name="y1")
y2 = model.addVar(lb=0, vtype=gb.GRB.CONTINUOUS, name="y2")
y3 = model.addVar(lb=0, vtype=gb.GRB.CONTINUOUS, name="y3")

# Set the objective function to minimize
model.setObjective(4*y1 + 13*y2 + 31*y3, gb.GRB.MINIMIZE)

# Add constraints
dual_constraint1 = model.addConstr(y1 + y2 + 5*y3 >= 5, "Dual_Constraint1")
dual_constraint2 = model.addConstr(2*y2 + 3*y3 >= 4, "Dual_Constraint2")

# Optimize the dual model
model.optimize()

# Check if the optimization was successful
if model.status == gb.GRB.OPTIMAL:
    # Get the optimal solution and objective value for the dual problem
    optimal_y1 = y1.x
    optimal_y2 = y2.x
    optimal_y3 = y3.x
    optimal_dual_objective_value = model.objVal

    # Print the results
    print("Optimal Dual Solution:")
    print(f"y1 = {optimal_y1}")
    print(f"y2 = {optimal_y2}")
    print(f"y3 = {optimal_y3}")
    print("Optimal Dual Objective Value:")
    print(f"Dual z = {optimal_dual_objective_value}")
    
    # These should equal the optimal solution to the primal problem
    print("Shadow Prices: ", (dual_constraint1.pi, dual_constraint2.pi))
else:
    print("No feasible solution found for the dual problem.")


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[x86] - Darwin 24.2.0 24C101)

CPU model: Intel(R) Core(TM) i5-8257U CPU @ 1.40GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2 rows, 3 columns and 5 nonzeros
Model fingerprint: 0x5fb558cc
Coefficient statistics:
  Matrix range     [1e+00, 5e+00]
  Objective range  [4e+00, 3e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+00, 5e+00]
Presolve time: 0.04s
Presolved: 2 rows, 3 columns, 5 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.625000e+00   0.000000e+00      0s
       2    3.5857143e+01   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.06 seconds (0.00 work units)
Optimal objective  3.585714286e+01
Optimal Dual Solution:
y1 = 0.0
y2 = 0.7142857142857144
y3 = 0.8571428571428571
Optimal Dual Objective Value:
Dual z = 35.85714285714286
Shadow Prices:  (3.2857142857142856, 4.857142857142858